# Script 27 full network nearest data

## Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from shapely.geometry import Point
import osmnx as ox

from tqdm import tqdm
import h3

import os
import sys
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

## Notebook config

In [13]:
gral_dir = '../../../data/external/santiago/'
all_pois_dir = gral_dir + "pois/"
output_dir = gral_dir + "nearest/"

projected_crs = "EPSG:32719"
source_list = ['carniceria','hogar','bakeries','supermercado','banco', #supplying-wellbeing
               #supplying-sociability
               'ferias','local_mini_market','correos', 
               #supplying-environmental impact
               'centro_recyc',

               #caring-wellbeing
               'hospital_priv','hospital_pub','clinica_priv','clinica_pub','farmacia','vacunatorio_priv','vacunatorio_pub','consult_ado_priv','consult_ado_pub','salud_mental','labs_priv','residencia_adumayor',
               #caring-sociability
               'eq_deportivo_priv','eq_deportivo_pub','club_deportivo',
               #caring-environmental impact [areal data: 'noise','temp']

               #living-wellbeing
               'civic_office','tax_collection','social_security','police','bomberos',
               #living-sociability [areal data: 'houses','social_viv','hotel']
               #living-environmental impact [areal_data: 'inter']
               
               #enjoying-wellbeing [areal data: 'ndvi']
               'museos_priv','museos_pub','cines','sitios_historicos',
               #enjoying-sociability
               'restaurantes_bar_cafe','librerias','ep_plaza_small',
               #enjoying-environmental impact
               'ep_plaza_big',

               #learning-wellbeing
               'edu_basica_pub','edu_media_pub','jardin_inf_pub','universidad', 'edu_tecnica',
               #learning-sociability
               'edu_adultos_pub','edu_especial_pub','bibliotecas',
               #learning-environmental impact
               'centro_edu_amb',

               #working-wellbeing
               'paradas_tp_ruta','paradas_tp_metro','paradas_tp_tren',
               #working-sociability [areal data: 'oficinas']
               #working-environmental impact
               'ciclovias','estaciones_bicicletas']

# Unique ID sources require extra code
unique_id_sources = ['ferias','ep_plaza_small','ep_plaza_big','ciclovias']
goi_id = 'ID'

## G, nodes and edges for full network

In [3]:
# Complete network - Load edges
compnet_edges = gpd.read_file(gral_dir+'calidad_ep/redvial2019_buffer_3750m_c_utilidad_2_single_parts.gpkg')
compnet_edges = compnet_edges.set_crs(projected_crs)
# Complete network - Load nodes
compnet_nodes = gpd.read_file(gral_dir +'calidad_ep/redvial2019_buffer_3750m_c_utilidad_2_nodes.shp')
compnet_nodes = compnet_nodes.set_crs(projected_crs)
# Complete network - Create navigable network
all_nodes, all_edges = aup.create_network(compnet_nodes, compnet_edges, projected_crs)
all_nodes = all_nodes.drop_duplicates(subset=['osmid'])
# Complete network - Create G
all_nodes_gdf = all_nodes.copy()
all_nodes_gdf.set_index('osmid',inplace=True)
all_edges_gdf = all_edges.copy()
all_edges_gdf.set_index(['u','v','key'],inplace=True)
all_nodes_gdf['x'] = all_nodes_gdf['geometry'].x
all_nodes_gdf['y'] = all_nodes_gdf['geometry'].y
# Complete network
full_G = ox.graph_from_gdfs(all_nodes_gdf, all_edges_gdf)
full_nodes = all_nodes_gdf.copy()
full_edges = all_edges_gdf.copy()

In [4]:
# Show
print(full_nodes.shape)
full_nodes.head(1)

(22562, 45)


,fid,IDcllbf,shp_lng,st_lng_,Nom_Rut,Rol_Mop,Clas_Rt,Tip_Crp,Catego,Cod_Rgn,...,buffer800,vertex_pos,vertex_ind,vertex_par,vertex_p_1,distance,angle,geometry,x,y
osmid,,,,,,,,,,,,,,,,,,,,,
3509586302268,1.0,1.0,0.0,1813.857525,Túnel Kennedy,None,5,None,Urbano,13,...,0.0,0,0,0,0,0.0,75.686533,POINT (-70.60280 -33.40858),-70.602804,-33.408582


In [5]:
# Show
print(full_edges.shape)
full_edges.head(1)

(31554, 37)


,,,IDcllbf,shp_lng,st_lng_,Nom_Rut,Rol_Mop,Clas_Rt,Tip_Crp,Catego,Cod_Rgn,id,...,ndvi_mn,ndvi_md,es_cll_,utilidd,rank_tl,pje_ep,UV,buffer800,geometry,length
u,v,key,,,,,,,,,,,,,,,,,,,,,
3509586302268,3512566302306,0,1.0,0.0,1813.857525,Túnel Kennedy,None,5,None,Urbano,13,1.0,...,0.121441,0.1365,1.0,0.184971,6596.0,0.209171,0,0,"LINESTRING (-70.60280 -33.40858, -70.60255 -33...",300.673461


## Load area of interest

In [6]:
aoi_schema = 'projects_research'
aoi_table = 'santiago_aoi'
city = 'alamedabuffer_4500m'
query = f"SELECT * FROM {aoi_schema}.{aoi_table} WHERE \"city\" LIKE \'{city}\'"
aoi = aup.gdf_from_query(query, geometry_col='geometry')
aoi = aoi.set_crs("EPSG:4326")

## Calculate and save nearest

In [14]:
for source in source_list:
    print(f"Calculating and saving nearest for {source}.")
    # Directory where pois to be examined are located
    pois_dir = all_pois_dir + f'{source}.gpkg'
    # Load all pois from directory
    pois = gpd.read_file(pois_dir)
    # Set pois CRS
    try:
        pois = pois.to_crs("EPSG:4326")
    except:
        pois = pois.set_crs("EPSG:4326")
    # Clip pois to area of interest
    source_pois = gpd.sjoin(pois, aoi)
    # Find nearest
    nearest = aup.find_nearest(full_G, full_nodes, source_pois, return_distance= True)
    nearest = nearest.set_crs("EPSG:4326")

    if source in unique_id_sources:
        # Group by node (osmid) and polygon (green space) considering only the closest vertex (min)
        groupby = nearest.groupby(['osmid',goi_id]).agg({'distance_node':np.min})
        
        # Turns back into gdf merging back with nodes geometry
        geom_gdf = full_nodes.reset_index()[['osmid','geometry']]
        groupby.reset_index(inplace=True)
        nearest = pd.merge(groupby,geom_gdf,on='osmid',how='left')
        nearest = gpd.GeoDataFrame(nearest, geometry="geometry")
        
        # Filters for pois assigned to nodes at a maximum distance of 80 meters (aprox. 1 minute)
        # That is to consider a 1 minute additional walk as acceptable (if goi is inside a park, e.g. a bike lane).
        nearest = nearest.loc[nearest.distance_node <= 80]
    
    # Save nearest
    nearest.to_file(output_dir + f"nearest_{source}.gpkg", driver='GPKG')

Calculating and saving nearest for carniceria.
Calculating and saving nearest for hogar.
Calculating and saving nearest for bakeries.
Calculating and saving nearest for supermercado.
Calculating and saving nearest for banco.
Calculating and saving nearest for ferias.
Calculating and saving nearest for local_mini_market.
Calculating and saving nearest for correos.
Calculating and saving nearest for centro_recyc.
Calculating and saving nearest for hospital_priv.
Calculating and saving nearest for hospital_pub.
Calculating and saving nearest for clinica_priv.
Calculating and saving nearest for clinica_pub.
Calculating and saving nearest for farmacia.
Calculating and saving nearest for vacunatorio_priv.
Calculating and saving nearest for vacunatorio_pub.
Calculating and saving nearest for consult_ado_priv.
Calculating and saving nearest for consult_ado_pub.
Calculating and saving nearest for salud_mental.
Calculating and saving nearest for labs_priv.
Calculating and saving nearest for resi

In [ ]:
nearest